In [23]:
import numpy as np

In [24]:
rho_tank=4500 #kg/m3
m=1020/3600#kg/s
P=172.37#bar
T=273.15+25
Tref=10+273.15
Po=31.026

In [25]:
P1=Po
P2=P
N_etapas=np.ceil((np.log(P2)/np.log(P1))/np.log(2))
N_etapas

np.float64(3.0)

In [26]:
def Cp_Cv(T):
        return 1.5949974007561318 * T ** (-0.021034016066268504)
def compressor_power(m_dot, T1, P_ratio, gamma, cp, eta):
    """
    Calcula la potencia requerida por un compresor para la compresión adiabática de un gas.

    Parámetros:
      m_dot : float
          Caudal másico del gas (kg/s).
      T1 : float
          Temperatura de entrada al compresor (K).
      P_ratio : float
          Relación de presión (P2 / P1, sin unidades) a la que se desea comprimir.
      gamma : float
          Relación de calores específicos (Cp/Cv).
      cp : float
          Calor específico a presión constante (J/kg·K).
      eta : float
          Eficiencia del compresor (valor entre 0 y 1).

    Retorna:
      power : float
          Potencia del compresor requerida (W, vatios).
    """
    # Cálculo del trabajo específico en J/kg
    specific_work = (cp * T1 * gamma / (gamma - 1)) * ((P_ratio)**((gamma - 1) / gamma) - 1)
    
    # Potencia requerida = caudal másico * trabajo específico, corregido por la eficiencia
    power = m_dot * specific_work / eta
    return power/1000

# Ejemplo de uso:
if __name__ == "__main__":
    # Parámetros de entrada (se pueden ajustar según el caso particular)
    T1      = 300           # Temperatura de entrada en Kelvin
    P_ratio = 10            # Relación de presiones (por ejemplo, de 1 a 10)
    gamma   = 1.40          # Relación de calores específicos (puede variar para hidrógeno, p.ej., ~1.40 en bajas temperaturas)
    cp      = 14300         # Calor específico de hidrógeno a presión constante (J/kg·K). Nota: El valor de hidrógeno es mayor debido a su baja masa molar.
    eta     = 0.75          # Eficiencia del compresor (75%)

    CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
    CP=CP/2*1000
    
    
    power_required = compressor_power(m, T, P_ratio, Cp_Cv(T), CP, eta)

In [27]:
import numpy as np
from scipy.optimize import root

def balance(x, T, Tref, m):
    global e
    global s
    """
    Calcula la función de balance para el proceso,
    devolviendo la diferencia e - s.

    Parámetros:
      x    : vector de incógnitas; se asume que x[0] es la variable a encontrar.
      T    : temperatura en K.
      Tref : temperatura de referencia en K.
      m    : masa del hidrógeno (kg).

    La función utiliza dos series polinómicas (para H2 y H2O)
    para calcular, respectivamente, las energías 'e' y 's', y retorna e - s.
    """
    # Calcular la entalpía (u otra propiedad) para hidrógeno
    Ta=15+273.15
    HhH2 = ( 19.67099783*(T - Tref)
              + (0.069681519/2) * (T**2 - Tref**2)
              - (0.000200098/3) * (T**3 - Tref**3)
              + (2.89493E-07/4) * (T**4 - Tref**4)
              - (2.22475E-10/5) * (T**5 - Tref**5)
              + (8.81466E-14/6) * (T**6 - Tref**6)
              - (1.42043E-17/7) * (T**6 - Tref**6)
            ) / 2
    e = HhH2 * m

    # Calcular la entalpía (u otra propiedad) para agua (H2O)
    HhH2O = ( -22.41701677*(Ta - Tref)
               + (0.876972156/2) * (Ta**2 - Tref**2)
               - (0.002570393/3) * (Ta**3 - Tref**3)
               + (2.48383E-06/4) * (Ta**4 - Tref**4)
             ) / 18.015
    s = HhH2O * x[0]
    
    return e - s


In [28]:
P_diff=P-Po
P_etapa=P_diff/N_etapas
Po=Po
Pf=P
P_acum=[Po]
Temps=[300]
Tref=10+273.15
Qs,Ts,m_H2O,comp,power,Ps=[],[],[],[],[],[]
for i in range(int(N_etapas)):
    P1=sum(P_acum)
    P_acum.append(P_etapa)
    P2=sum(P_acum)
    Z = 0.99704 + P * (6.4149e-9)
    V = ((m)/(2)*8.314*Temps[-1])/sum(P_acum)
    V2=V*(P1/P2)**(1/Cp_Cv(Temps[-1]))
    Z = 0.99704 + P2 * 1e6 * (6.4149e-9)
    Ti=(sum(P_acum)*V2*2)/(m*Z*8.314)
    print('T',Ti)
    # Valor inicial para la incógnita x (en este caso, es un vector de una dimensión)
    x0 = [1.0]

    # Se utiliza scipy.optimize.root con método 'hybr'
    solucion = root(balance, x0, args=(25+273.15, Ti, m), method='hybr', tol=1e-1)

    if solucion.success:
        x_encontrado = solucion.x[0]
        print(f"Solución encontrada: x = {x_encontrado:.6f}")
        Temps.append(25+273.15)
        Qs.append(float(e))
        Ts.append((25+273.15))
        m_H2O.append(float(x_encontrado))
        comp.append('No. '+str(i+1))
        CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
        CP=CP/2*1000
        power.append(float(compressor_power(m, Ti, P2/P1, Cp_Cv(Ti), CP, 0.75)))
        Ps.append(P2)
    else:
        print("No se encontró solución:", solucion.message)
        

T 104.22130884981007
Solución encontrada: x = 1.110899
T 118.63099642179822
Solución encontrada: x = 1.095111
T 113.14492388617285
Solución encontrada: x = 1.100615


In [29]:
import pandas as pd
# Crear un DataFrame con los datos
data = {
	"Compressor": comp,
    "T (K)": Ts,
    "P (bar)": Ps,  # Convertir de Pa a bar
	"Cooling H2O (kg/s)": m_H2O,
	"Q (kW)": Qs,
    "W (kW)": power,
    "Q + W (kW)": np.array(Qs)+ np.array(power)
}

df_compresores = pd.DataFrame(data)
df_compresores.loc[df_compresores.shape[0]] = ["Total", Temps[-1], Ps[-1], sum(m_H2O), sum(Qs), sum(power), sum(Qs)+ sum(power)]

# Mostrar la tabla
df_compresores

,Compressor,T (K),P (bar),Cooling H2O (kg/s),Q (kW),W (kW),Q + W (kW)
0,No. 1,298.15,78.140667,1.110899,752.328341,605.382604,1357.710945
1,No. 2,298.15,125.255333,1.095111,700.647672,327.322829,1027.970501
2,No. 3,298.15,172.370000,1.100615,720.443502,206.290561,926.734063
3,Total,298.15,172.370000,3.306625,2173.419514,1138.995994,3312.415508
